In [1]:
import sys

sys.path.append('..')

In [2]:
from filter_clinvar_xml import filter_xml, pprint
from cmat.clinvar_xml_io import *
from cmat.clinvar_xml_io.clinical_classification import MultipleClinicalClassificationsError
from cmat.output_generation import consequence_type as CT
from cmat.output_generation.clinvar_to_evidence_strings import get_consequence_types
from cmat.clinvar_xml_io.xml_parsing import *

In [3]:
from opentargets_pharmgkb.evidence_generation import *
from opentargets_pharmgkb.pandas_utils import read_tsv_to_df

In [24]:
import pandas as pd
from collections import Counter

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

### Prepare data

In [6]:
data_dir = '/home/april/projects/opentargets/drug-response'

In [ ]:
# Download fresh ClinVar data
!wget -O {data_dir}/clinvar.xml.gz https://ftp.ncbi.nlm.nih.gov/pub/clinvar/xml/RCV_release/ClinVarRCVRelease_00-latest.xml.gz

In [ ]:
# Download fresh PGKB data
!wget -q https://api.pharmgkb.org/v1/download/file/data/clinicalAnnotations.zip
!wget -q https://api.pharmgkb.org/v1/download/file/data/variants.zip
!wget -q https://api.pharmgkb.org/v1/download/file/data/relationships.zip

!unzip -j clinicalAnnotations.zip "*.tsv" -d {data_dir}
!unzip -j variants.zip "*.tsv" -d {data_dir}
!unzip -j relationships.zip "*.tsv" -d {data_dir}
!rm clinicalAnnotations.zip variants.zip relationships.zip

In [7]:
drug_response_xml = f'{data_dir}/drug-response.xml.gz'

In [7]:
def drug_response(x):
    try:
        return 'drug response' in x.clinical_significance_list or x.trait_set_type == 'DrugResponse'
    except MultipleClinicalClassificationsError:
        return False

In [8]:
filter_xml(
    input_xml=f'{data_dir}/clinvar.xml.gz',
    output_xml=drug_response_xml,
    filter_fct=drug_response,
    max_num=None
)

INFO:filter_clinvar_xml:Records written: 5148


### Compare genes

#### ClinVar genes

First run the annotation pipeline on the drug response XML, then can use the combined consequences file.

In [ ]:
# Had to resume a couple times, output removed
!nextflow run ../../pipelines/annotation_pipeline.nf --output_dir {data_dir} --clinvar {drug_response_xml} -resume

In [8]:
# Map from variant IDs from ClinVar (can be RCV) to genes & consequence terms
clinvar_variant_to_gene_mappings = CT.process_consequence_type_file(f'{data_dir}/gene_mapping/consequences_combined.tsv')

INFO:cmat.output_generation:Loading mapping rs -> ENSG/SOterms
INFO:cmat.output_generation:2253 rs->ENSG/SOterms mappings loaded


In [9]:
# All clinvar genes as a set
clinvar_genes = {
    conseq.ensembl_gene_id
    for k, v in clinvar_variant_to_gene_mappings.items()
    for conseq in v
}

In [10]:
len(clinvar_genes)

154

In [44]:
len(clinvar_variant_to_gene_mappings)

2253

#### PGKB genes

In [9]:
fasta_path = '/home/april/projects/opentargets/pharmgkb/assembly/GCF_000001405.40_GRCh38.p14_genomic.fna'

# Load tables
clinical_annot_path = os.path.join(data_dir, 'clinical_annotations.tsv')
clinical_alleles_path = os.path.join(data_dir, 'clinical_ann_alleles.tsv')
clinical_evidence_path = os.path.join(data_dir, 'clinical_ann_evidence.tsv')
variants_path = os.path.join(data_dir, 'variants.tsv')
relationships_path = os.path.join(data_dir, 'relationships.tsv')

clinical_annot_table = read_tsv_to_df(clinical_annot_path)
clinical_alleles_table = read_tsv_to_df(clinical_alleles_path)
clinical_evidence_table = read_tsv_to_df(clinical_evidence_path)
variants_table = read_tsv_to_df(variants_path)
relationships_table = read_tsv_to_df(relationships_path)

In [51]:
len(clinical_annot_table)

5176

In [ ]:
# Main processing - derived from evidence generation pipeline
merged_with_variants_table = pd.merge(clinical_annot_table, variants_table, left_on='Variant/Haplotypes',
                                      right_on='Variant Name', how='left')
merged_with_alleles_table = pd.merge(merged_with_variants_table, clinical_alleles_table, on=ID_COL_NAME, how='left')

with_rs, no_rs = split_df_with_or_without_rs(merged_with_alleles_table)

# BRANCH 1: For rsIds, form genotype IDs + get consequences from VEP
genotype_ids_table = get_genotype_ids(with_rs, fasta_path)
rs_consequences_table = get_functional_consequences(genotype_ids_table)

# BRANCH 2: For named alleles, form haplotype IDs + map PGKB genes with Biomart
haplotype_ids_table = get_haplotype_ids(no_rs, relationships_table)

In [13]:
# debug dump
rs_consequences_table.to_csv(f'{data_dir}/pgkb_rs_consequences.csv')

In [17]:
# Rerun after biomart back up
no_rs_genes_table = explode_and_map_genes(haplotype_ids_table)

# debug dump
no_rs_genes_table.to_csv(f'{data_dir}/pgkb_star_consequences.csv')

INFO:cmat.consequence_prediction.common.biomart:Processing chunk of 35 records


In [13]:
# To load from file instead of re-running
rs_consequences_table = pd.read_csv(f'{data_dir}/pgkb_rs_consequences.csv')
no_rs_genes_table = pd.read_csv(f'{data_dir}/pgkb_star_consequences.csv')

In [14]:
# Merge the tables - will fill in NaNs where columns are missing
consequences_table = pd.concat((rs_consequences_table, no_rs_genes_table))

In [15]:
# Get relevant columns - genotype IDs, haplotype IDs, genes from VEP and genes from PGKB
pgkb_consequences_table = consequences_table[[ID_COL_NAME, 'Variant/Haplotypes', 'genotype_id', 'overlapping_gene', 'consequence_term', 'haplotype_id', 'gene_from_pgkb']]

In [16]:
# All PGKB genes as a set
pgkb_genes = set(pgkb_consequences_table.overlapping_gene.dropna().to_list()) | set(pgkb_consequences_table.gene_from_pgkb.dropna().to_list())

In [17]:
len(pgkb_genes)

1135

In [43]:
pgkb_consequences_table.shape

(21870, 7)

#### Comparison

In [19]:
genes_in_clinvar_not_pgkb = clinvar_genes - pgkb_genes

In [20]:
genes_in_pgkb_not_clinvar = pgkb_genes - clinvar_genes

In [49]:
len(genes_in_clinvar_not_pgkb)

75

In [50]:
len(genes_in_pgkb_not_clinvar)

1056

In [107]:
genes_in_clinvar_not_pgkb

{'ENSG00000007372',
 'ENSG00000050767',
 'ENSG00000066468',
 'ENSG00000099810',
 'ENSG00000099889',
 'ENSG00000100342',
 'ENSG00000100416',
 'ENSG00000103197',
 'ENSG00000104388',
 'ENSG00000105186',
 'ENSG00000108370',
 'ENSG00000108947',
 'ENSG00000109911',
 'ENSG00000110711',
 'ENSG00000115756',
 'ENSG00000116218',
 'ENSG00000117713',
 'ENSG00000129214',
 'ENSG00000129244',
 'ENSG00000129245',
 'ENSG00000130402',
 'ENSG00000131759',
 'ENSG00000136231',
 'ENSG00000136813',
 'ENSG00000137672',
 'ENSG00000138138',
 'ENSG00000140538',
 'ENSG00000140650',
 'ENSG00000141499',
 'ENSG00000141504',
 'ENSG00000146592',
 'ENSG00000147883',
 'ENSG00000147889',
 'ENSG00000151490',
 'ENSG00000151952',
 'ENSG00000157404',
 'ENSG00000157483',
 'ENSG00000157764',
 'ENSG00000163029',
 'ENSG00000168546',
 'ENSG00000172037',
 'ENSG00000173597',
 'ENSG00000176399',
 'ENSG00000176463',
 'ENSG00000177084',
 'ENSG00000178295',
 'ENSG00000178966',
 'ENSG00000181555',
 'ENSG00000182185',
 'ENSG00000183018',


In [61]:
# Find the ClinVar record associated with ZFP36L1 (one gene in clinvar not in pgkb)
for clinvar_set in ClinVarDataset(drug_response_xml).iter_cvs():
    if clinvar_set.rcv.accession == 'RCV000626430':
        break

In [64]:
pprint(clinvar_set.rcv.traits[0].trait_xml)

<Trait ID="37030" Type="DrugResponse">
        <Name>
          <ElementValue Type="Preferred">Poly (ADP-Ribose) polymerase inhibitor response</ElementValue>
        </Name>
        <Name>
          <ElementValue Type="Alternate">PARP Inhibitor response</ElementValue>
        </Name>
        <XRef ID="CN322715" DB="MedGen" />
      </Trait>
    


In [67]:
pprint(clinvar_set.scvs[0].traits[0].trait_xml)

<Trait Type="DrugResponse">
        <Name>
          <ElementValue Type="Preferred">PARP Inhibitor response</ElementValue>
        </Name>
        <TraitRelationship Type="DrugResponseAndDisease">
          <Name>
            <ElementValue Type="Preferred">Cancer</ElementValue>
          </Name>
        </TraitRelationship>
      </Trait>
    


In [11]:
drug_response_dataset = ClinVarDataset(drug_response_xml)

In [25]:
# Get number of records (RCVs) associated with genes not in PGKB
rcvs_for_genes_not_in_pgkb = []
categories = Counter()
for record in drug_response_dataset:
    if record.measure:
        consequences, category = get_consequence_types(record.measure, clinvar_variant_to_gene_mappings)
        genes = {conseq.ensembl_gene_id for conseq in consequences}
        if genes & genes_in_clinvar_not_pgkb:
            rcvs_for_genes_not_in_pgkb.append(record.accession)
            categories[category] += 1

In [26]:
categories

Counter({'SIMPLE': 346, 'COMPLEX': 3})

In [27]:
len(rcvs_for_genes_not_in_pgkb)

349

In [110]:
445/5148

0.08644133644133645

* Drug response records in ClinVar (RCVs): 5148
* Clinical annotations in PGKB: 5176
* Genes in ClinVar: 154
* Genes in PGKB: 1135
* Genes in ClinVar that are not in PGKB: 75
    * Associated RCVs: 349

In summary, `75/154 = 48.7%` of target genes covered by ClinVar drug response records are not present in PGKB. But this corresponds to only `349/5148 = 6.8%` of all drug response records in ClinVar.

Example: [ZFP36L1](https://www.ensembl.org/Homo_sapiens/Gene/Summary?g=ENSG00000185650;r=14:68787660-68796253)
* No data in [PGKB](https://www.pharmgkb.org/gene/PA35027/overview)
* Record in [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/variation/523157/)

Another example: [DIO1](https://www.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000211452;r=1:53891239-53911086)
* Variant annotations but NOT clinical annotations in [PGKB](https://www.pharmgkb.org/gene/PA27337/overview)
* Record in [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/variation/2573328/)

### PGKB/ClinVar overlap

In [29]:
pgkb_subs_in_clinvar = []
for record in drug_response_dataset.iter_cvs():
    if any('pharmgkb' in scv.submitter.lower() for scv in record.scvs):
        pgkb_subs_in_clinvar.append(record.rcv)

In [30]:
len(pgkb_subs_in_clinvar)

445

In [101]:
# Records without measure really means the measure type is not "Variant" in ClinVar
pgkb_records_without_measure = [record for record in pgkb_subs_in_clinvar if record.measure is None]

In [102]:
len(pgkb_records_without_measure)

0

In [163]:
# Check for PGKB xref
rcv_to_caid = {}
for record in pgkb_subs_in_clinvar:
    elts = find_elements(record.measure.measure_xml, './XRef[@DB="PharmGKB Clinical Annotation"]')
    if elts:
        rcv_to_caid[record.accession] = [elt.attrib['ID'] for elt in elts]

In [164]:
len(rcv_to_caid)

78

In [165]:
rcv_to_caid

{'RCV000211406': ['655385024'],
 'RCV000211192': ['1183888959', '981239773'],
 'RCV000211273': ['982035703'],
 'RCV000211316': ['1183888959', '981239773'],
 'RCV000211369': ['655385028'],
 'RCV001787697': ['1183491425', '1183491446'],
 'RCV000211256': ['1183960780'],
 'RCV000211326': ['981202609'],
 'RCV000211346': ['1183960766'],
 'RCV000211255': ['1183960790'],
 'RCV000211163': ['982030805'],
 'RCV001787915': ['981201535'],
 'RCV001787916': ['981201535'],
 'RCV001788073': ['982030805'],
 'RCV000211190': ['982030369'],
 'RCV000211129': ['1183960775'],
 'RCV000211144': ['1184661194', '1184661207', '655385400'],
 'RCV000211147': ['1183704228', '655385392'],
 'RCV000211149': ['613978931', '613978937'],
 'RCV000211150': ['1183960795'],
 'RCV000211178': ['1183492249'],
 'RCV000211184': ['1183888969', '827862764'],
 'RCV000211242': ['655384799'],
 'RCV000211246': ['1183960805'],
 'RCV000211264': ['1183960318'],
 'RCV000211275': ['1183704228', '655385392'],
 'RCV000211278': ['1445585748', '6

In [38]:
pgkb_ids_in_clinvar = [record.measure.vcf_full_coords for record in pgkb_subs_in_clinvar
                       if record.measure is not None and record.measure.has_complete_coordinates]

In [39]:
len(pgkb_ids_in_clinvar)

445

In [40]:
uniq_pgkb_ids_in_clinvar = set(pgkb_ids_in_clinvar)

In [42]:
len(uniq_pgkb_ids_in_clinvar)

136

* 445 PGKB-submitted records in ClinVar
* _All_ of them are associated with single variants!
  * Some are marked as "part of a haplotype" - e.g. [here](https://www.ncbi.nlm.nih.gov/clinvar/variation/17848/)
* Cannot find any examples of RCVs in ClinVar supported by PGKB star allele clinical annotations - hard to tell for certain but I think this means star allele clinical annotations are not submitted to ClinVar, even levels 1/2
    * [example 1](https://www.pharmgkb.org/clinicalAnnotation/1451241420) - [clinvar search](https://www.ncbi.nlm.nih.gov/clinvar/?term=NAT2%5Bgene%5D+AND+PharmGKB%5Bsubmitter%5D)
    * [example 2](https://www.pharmgkb.org/clinicalAnnotation/1183621000) - [clinvar search](https://www.ncbi.nlm.nih.gov/clinvar/?term=G6PD%5Bgene%5D+AND+PharmGKB%5Bsubmitter%5D)
* Explicit cross-reference to PGKB from ClinVar sometimes exist but not always

Also, only `445/5148 = 8.6%` of ClinVar's drug response records come from PGKB - but this corresponds to about 50% of the target genes.

### Compare variants

In [103]:
records_without_measure = [record for record in drug_response_dataset if record.measure is None]

In [104]:
len(records_without_measure)

2469

In [111]:
2469/5148

0.4796037296037296

Drug response records in ClinVar are enriched in non-single variant measures, this number is <1% for ClinVar as a whole. Will look at variants where we do parse them, keeping in mind this will only be about half the data.

#### Coordinate ids (chr_pos_ref_alt)

In [43]:
clinvar_variant_ids = {k for k in clinvar_variant_to_gene_mappings if (not k.startswith('RCV') and not k.startswith('NC'))}

In [44]:
len(clinvar_variant_ids)

2247

In [45]:
pgkb_variant_ids = [vid.split(' ') for gid in pgkb_consequences_table.genotype_id.dropna() for vid in genotype_id_to_vep_ids(gid)]
pgkb_variant_ids = {':'.join((vid[0], vid[1], vid[3], vid[4])) for vid in pgkb_variant_ids}

In [46]:
len(pgkb_variant_ids)

2864

In [47]:
variant_ids_in_clinvar_not_pgkb = clinvar_variant_ids - pgkb_variant_ids
variant_ids_in_pgkb_not_clinvar = pgkb_variant_ids - clinvar_variant_ids

In [48]:
len(variant_ids_in_clinvar_not_pgkb)

2055

In [49]:
len(variant_ids_in_pgkb_not_clinvar)

2672

In [146]:
2055/2247

0.9145527369826435

In [51]:
# These are variant IDs from PGKB submissions _in ClinVar_ that we are not able to find as variant IDs _in PGKB_
variant_ids_in_clinvar_not_pgkb & {i.replace('_', ':') for i in uniq_pgkb_ids_in_clinvar}

{'17:63488529:T:TATACAGTCACTTTTTTTTTTTTTTTGAGACGGAGTCTCGCTCTGTCGCCC',
 '19:38499644:TGGA:T',
 '1:97450065:TG:T',
 '7:117559590:ATCT:A',
 'MT:1095:T:C',
 'MT:1494:C:T',
 'MT:1555:A:G'}

In [52]:
129 / 136

0.9485294117647058

* Variants in ClinVar: 2247
* Variants in PGKB: 2864
* Variants in ClinVar that are not in PGKB: 2055 (91.5%)

This _might_ not be 100% accurate since the computation of the IDs is done slightly differently (rsID would be better), but checked a couple cases.

Example variant: rs934034534
* In [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/variation/828943/)
* Not in [PGKB](https://www.pharmgkb.org/search?query=rs934034534)

Example 2: rs767351070
* In [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/variation/881532/)
* Not in [PGKB](https://www.pharmgkb.org/search?query=rs767351070)

This also ignores cases where a named allele corresponds to a single rsID, e.g. [CYP2C9*2](https://www.pharmgkb.org/haplotype/PA165816543). In other words, we would need to decompose star alleles to determine the true variant overlap.

#### Other ids (repeats and structural in Clinvar, star alleles in PGKB)

In [147]:
weird_clinvar_variant_ids = {k for k in clinvar_variant_to_gene_mappings if (k.startswith('RCV') or k.startswith('NC'))}

In [148]:
weird_clinvar_variant_ids

{'NC_000009.12:g.21879075_22096084del',
 'NC_000009.12:g.21939409_22706614del',
 'NC_000010.11:g.87830830_88617225del',
 'NC_000014.9:g.67823656_68813174del',
 'NC_000017.11:g.7603519_7768486del',
 'RCV003317021'}

In [150]:
pgkb_haplotype_ids = set(pgkb_consequences_table.haplotype_id.dropna())

In [ ]:
pgkb_haplotype_ids

* [RCV003317021](https://www.ncbi.nlm.nih.gov/clinvar/RCV003317021/) is a repeat on UGT - could be part of a [PGKB star allele](https://www.pharmgkb.org/gene/PA420/haplotype)
* For the others, it's really tricky to determine - again decomposing star alleles could help